# Data Prep Code

This code prepares all of the data that the dashboard code uses as input.

In [1]:
#import libraries
from itertools import product
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, functions
import pyspark.sql.functions as f
from pyspark.sql.types import *

import geopandas
from geojson_rewind import rewind
import json

The next cell defines helper functions used elsewhere in the notebook.

In [2]:
#function for executing a query via spark
def query_to_df(query, show_num = False, count = False, export = False):
    
    #run query
    result = spark.sql(query)
    
    #show show_num rows of the data if applicable
    if show_num:
        result.cache()
        result.show(show_num)
    
    #show the count (number of rows) if applicable
    if count:
        result.cache()
        n_rows = result.count()
        print('Rows:',n_rows)
        
    #store in df
    result_df = result.toPandas()
    
    #export df to csv if applicable
    if export:
        result_df.to_csv(export, index = False)
        
    #return df
    return result_df


#function for formatting brand names
special_fmt_brands = {'gmc':'GMC',
                      'bmw':'BMW',
                      'mercedesbenz':'Mercedes-Benz'}
def get_formatted_brand_name(x):
    if x.lower() in special_fmt_brands:
        return special_fmt_brands[x.lower()]
    else:
        return x.title()

The below cell specifies the latest year and month as far as the analysis is concerned.

In [3]:
#specify current year
curr_yr = 2023
#specify current month
curr_mo = 9

## Read in Data

Next, we read in supplementary files related to county median incomes, populations, and areas (to get populatin density), as well as a geographic file that has unique numeric IDs for all of the counties. All of this data is merged together into the `map_df` data frame. This data will be needed for map and scatter plot visuals in the dashboard. 

In [4]:
#read in and pre-process median income data
medinc_df = pd.read_csv('raw_data/medinc2020.csv').rename(columns = {'Label (Grouping)':'county',
                'Median family income in the past 12 months (in 2020 inflation-adjusted dollars)':'medinc'})
medinc_df['medinc'].fillna(method = 'backfill', inplace = True)
medinc_df['medinc'] = medinc_df['medinc'].str.replace(r',|±','', regex = True).astype(int)
medinc_df = medinc_df[~(medinc_df['county'].str.strip().isin({'Estimate','Margin of Error'}))]\
                     .reset_index(drop = True)
medinc_df['county'] = medinc_df['county'].str.replace('County, Washington','').str.strip()

#read in and pre-process median income data
population_df = pd.read_csv('raw_data/population2020.csv').rename(columns = {'Label (Grouping)':'county',
                                                                         'Total':'population'})
population_df['population'].fillna(method = 'backfill', inplace = True)\
                   
population_df['population'] = population_df['population'].str.replace(r',','', regex = True)
population_df = population_df[~(population_df['county'].str.strip().isin({'Estimate','Margin of Error'}))]\
                             .reset_index(drop = True)
population_df['population'] = population_df['population'].astype(int) 
population_df['county'] = population_df['county'].str.replace('County, Washington','').str.strip()

#read in and pre-process area data
area_df = pd.read_csv('raw_data/county_area.csv')

#merge into demographic df
demo_df = medinc_df.merge(population_df, how = 'inner', on = 'county')\
                   .merge(area_df, how = 'inner', on = 'county')

#add population density variable
demo_df['popdensity'] = (demo_df['population']/demo_df['area']).round(2)

#read in geo df
#geojson data
geojson_path = 'raw_data/geo_data.geojson'
with open(geojson_path,'r') as f:
    counties = json.load(f)
#county ids
county_id_map = {}
for i,feature in enumerate(counties['features']):
    id_num = feature['properties']['OBJECTID']
    county_name = feature['properties']['JURISDICT_LABEL_NM']
    feature['id'] = id_num
    county_id_map[county_name] = id_num
#rewind geojson
counties = rewind(counties,rfc7946 = False)

#geopandas
geojson_path = 'raw_data/geo_data.geojson'
gdf = gpd.read_file(geojson_path)

#ensure id matches geojson
gdf['id'] = gdf['JURISDICT_LABEL_NM'].apply(lambda x:  county_id_map[x])
gdf = gdf[['JURISDICT_LABEL_NM','id']].rename(columns = {'JURISDICT_LABEL_NM':'county'})

#join demo_df with geo id
map_df = gdf.merge(demo_df, how = 'left', on = 'county')

print('map_df:')
display(map_df.head(5))
print('...')

map_df:


,county,id,medinc,population,area,popdensity
0,Grant,2455,67262,96648,2680,36.06
1,Benton,2459,84750,200715,1700,118.07
2,Okanogan,4179,55779,42080,5266,7.99
3,Ferry,4183,63490,7643,2203,3.47
4,Garfield,6623,68355,2258,711,3.18


...


The next cell instantiates a Spark session to help us work with the registration data, which is a larger data set.

In [6]:
#instantiate spark session
spark = SparkSession.builder\
                    .appName('vr')\
                    .config('spark.cores.max','4')\
                    .getOrCreate()

Then, we read in the vehicle registration data using Spark, rename some of the columns, and register it as the table `raw_data`, so that we can query the data using SQL. We also print the number of rows in the data.

In [7]:
#read in data
df = spark.read.csv('raw_data/reg_data.csv', header = True)

#rename columns
col_rename_dict = {'Transaction Month and Year':'tr_mo_yr',
                  'Make':'brand',
                  'Model':'model',
                  'Model Year':'v_year',
                  'Vehicle Type':'vehicle_type',
                  'Vehicle Primary Use':'vehicle_primary_use',
                  'Fuel Type Primary':'fuel_type_1',
                  'Fuel Type Secondary':'fuel_type_2',
                  'Transaction Type':'transact_type',
                  'Owner Type':'own_type',
                  'County':'county',
                  'State':'state',
                  'Postal Code':'post_code'}
for key,val in col_rename_dict.items():
    df = df.withColumnRenamed(key,val)

print('Rows:','{:,.0f}'.format(df.count()))
    
#register as table
df.createOrReplaceTempView('raw_data')

Rows: 24,990,778


The following cell performs some basic preprocessing. Specifically, it creates transaction `month` and `year` variables based on the `tr_mo_yr` column, drops hyphens and spaces from the brand names, subsets relevant variables, and filters out rows where the transaction type is not "Original Registration." This is because the dashboard will only include original registrations.

It also prints the number of rows that remain after this filtering, and stores the results in a table `sub_data`.

In [8]:
#select relevant columns and subset to include only original registrations
query = """SELECT CAST(SUBSTRING(tr_mo_yr,1,2) AS INT) AS month,
                  CAST(SUBSTRING(tr_mo_yr,7,4) AS INT) AS year, 
                  REPLACE(REPLACE(brand,'-',''),' ','') AS brand, 
                  model, v_year, own_type, county, 
                  fuel_type_1, fuel_type_2,
                  state, post_code, transact_type
                      FROM raw_data
                  WHERE transact_type = 'Original Registration'"""
subdata = spark.sql(query)
#preview
subdata.show(3)
n = subdata.count()
print('Rows kept:','{:,.0f}'.format(n))
#register as table
subdata.createOrReplaceTempView('sub_data')

+-----+----+-----+-----+------+----------------+------+-----------+-----------+-----+---------+--------------------+
|month|year|brand|model|v_year|        own_type|county|fuel_type_1|fuel_type_2|state|post_code|       transact_type|
+-----+----+-----+-----+------+----------------+------+-----------+-----------+-----+---------+--------------------+
|    1|2020|ACURA|  RDX|  2018|Individual Owner|Pierce|   Gasoline|       null|   WA|    98335|Original Registra...|
|    1|2020|ACURA|  RDX|  2020|Individual Owner|  King|   Gasoline|       null|   WA|    98033|Original Registra...|
|    1|2020|ACURA|   TL|  2006|Individual Owner|Benton|   Gasoline|       null|   WA|    99354|Original Registra...|
+-----+----+-----+-----+------+----------------+------+-----------+-----------+-----+---------+--------------------+
only showing top 3 rows



Rows kept: 1,826,218


## Create Fuel Types

The original dataset includes primary and seconday fuel type fields &mdash; `fuel_type_1` and `fuel_type_2`, respectively. The below SQL query produces a table that shows the counts and percentage share of each combination of these fields.

In [9]:
#primary-secondary fuel type combos counts/percentages
query = f"""SELECT fuel_type_1, fuel_type_2, COUNT(*) AS count, ROUND(100*COUNT(*)/{n},4) AS pct 
            FROM sub_data
            GROUP BY fuel_type_1,fuel_type_2
            ORDER BY count DESC"""
result = spark.sql(query)
result.show(100)

+--------------------+--------------------+-------+-------+
|         fuel_type_1|         fuel_type_2|  count|    pct|
+--------------------+--------------------+-------+-------+
|            Gasoline|                null|1405189|76.9453|
|            Gasoline|            Electric| 137027| 7.5033|
|            Electric|                null|  92253| 5.0516|
|              Diesel|                null|  73108| 4.0032|
|Flexible Fuel Veh...|                null|  57017| 3.1221|
|            Gasoline|       Ethanol (E85)|  22584| 1.2367|
|            Electric|            Gasoline|  20454|   1.12|
|            Gasoline|Flexible Fuel Veh...|   7437| 0.4072|
|Flexible Fuel Veh...|       Ethanol (E85)|   7392| 0.4048|
|Flexible Fuel Veh...|Flexible Fuel Veh...|   1282| 0.0702|
|Flexible Fuel Veh...|            Gasoline|    963| 0.0527|
|Flexible Fuel Veh...|            Electric|    477| 0.0261|
|            Gasoline|            Gasoline|    331| 0.0181|
|            Gasoline|Compressed Natura.

To simplify this, we will convert these values into a single `fuel_cat` field, based on the following rules:

 - "Gasoline" vehicles will be those with a primary fuel type of "Gasoline" and a secondary fuel type of NULL, "Gasoline," "Ethanol (E85)," or "Flexible Fuel Vehicle (FFV)"; 
 - "Hybrid" vehicles will be those with a primary fuel type of "Hybrid" and any secondary fuel type, or a primary fuel type of "Gasoline" and a secondary fuel type of "Electric," or a primary fuel type of "Electric" and a secondary fuel type of "Gasoline"; 
 - "Electric" vehicles will be those with a primary fuel type of "Electric" and a secondary fuel type of NULL; 
 - "Diesel" vehicles will be those with a primary fuel type of "Diesel" and a secondary fuel type of NULL; 
 - The "Other" category consists of all vehicles that do not meet any of these definitions.
 
This is implemented in the below SQL query, and the table with the new `fuel_cat` variable is registered as `data`.

In [10]:
#create single fuel category based on combinations of fuel types 1,2
query = """
SELECT *,
    CASE WHEN fuel_cat='Gasoline' THEN 1
      ELSE 0
      END AS gasoline,
    CASE WHEN fuel_cat='Hybrid' THEN 1
      ELSE 0
      END AS hybrid,
    CASE WHEN fuel_cat='Electric' THEN 1
      ELSE 0
      END AS electric,
    CASE WHEN fuel_cat='Diesel' THEN 1
      ELSE 0
      END AS diesel,
    CASE WHEN fuel_cat='Other' THEN 1
      ELSE 0
      END AS other
FROM
    (SELECT *, 
        CASE
            WHEN fuel_type_1 = 'Gasoline' AND (fuel_type_2 IS NULL OR fuel_type_2 IN ('Gasoline','Ethanol (E85)','Flexible Fuel Vehicle (FFV)')) 
                THEN 'Gasoline'
            WHEN fuel_type_1 = 'Electric' AND (fuel_type_2 IS NULL) 
                THEN 'Electric'
            WHEN fuel_type_1 = 'Hybrid' OR (fuel_type_1 = 'Gasoline' AND fuel_type_2 = 'Electric') OR (fuel_type_1 = 'Electric' AND fuel_type_2 = 'Gasoline')
                THEN 'Hybrid'
            WHEN fuel_type_1 = 'Diesel' AND (fuel_type_2 IS NULL)
                THEN 'Diesel'
            ELSE 'Other' 
        END AS fuel_cat
    FROM sub_data)
"""
#run query
subdata = spark.sql(query)

#preview
subdata.show(3)

#register as table
subdata.createOrReplaceTempView('data')

+-----+----+-----+-----+------+----------------+------+-----------+-----------+-----+---------+--------------------+--------+--------+------+--------+------+-----+
|month|year|brand|model|v_year|        own_type|county|fuel_type_1|fuel_type_2|state|post_code|       transact_type|fuel_cat|gasoline|hybrid|electric|diesel|other|
+-----+----+-----+-----+------+----------------+------+-----------+-----------+-----+---------+--------------------+--------+--------+------+--------+------+-----+
|    1|2020|ACURA|  RDX|  2018|Individual Owner|Pierce|   Gasoline|       null|   WA|    98335|Original Registra...|Gasoline|       1|     0|       0|     0|    0|
|    1|2020|ACURA|  RDX|  2020|Individual Owner|  King|   Gasoline|       null|   WA|    98033|Original Registra...|Gasoline|       1|     0|       0|     0|    0|
|    1|2020|ACURA|   TL|  2006|Individual Owner|Benton|   Gasoline|       null|   WA|    99354|Original Registra...|Gasoline|       1|     0|       0|     0|    0|
+-----+----+----

## Identify Top Brands from the Current Year

Some of the visuals will include data specific to certain car brands, and will allow the user to specify a brand. We will limit these options to the top 15 brands for the current year. The below cell identifies and stores these brands, both in data frame and a string that can be used in SQL queries. 

In [11]:
#get top brands for most recent year only
num_brands = 15
query = f"""
SELECT brand, count
FROM
(SELECT brand, COUNT(*) AS count
    FROM data
    WHERE year={curr_yr}
    GROUP BY brand
    ORDER BY -count)
LIMIT {num_brands}
"""
top_brands_curryr = query_to_df(query, show_num = num_brands)
top_brands_curryr_str = str(tuple(top_brands_curryr['brand']))

+------------+-----+
|       brand|count|
+------------+-----+
|      TOYOTA|56984|
|       HONDA|33177|
|        FORD|32515|
|      SUBARU|29383|
|   CHEVROLET|24494|
|       TESLA|18352|
|         KIA|16332|
|      NISSAN|15583|
|     HYUNDAI|15430|
|        JEEP|12783|
|       MAZDA|11885|
|         BMW|10344|
|  VOLKSWAGEN| 8694|
|         GMC| 7856|
|MERCEDESBENZ| 7033|
+------------+-----+



## Card Data

The next several cells process data that will be featured in indicator cards at the top of the dashboard.

This cell gets the overall registration volume for the current year, as well as for the previous year, so that a year-over-year percent change can be shown.

In [12]:
query = f"""
SELECT year, COUNT(*) AS count
FROM data
WHERE year IN ({curr_yr-1},{curr_yr}) AND month <= {curr_mo}
GROUP BY year
ORDER BY year
"""
total_registration_counts_2y = query_to_df(query, show_num = 2, 
                                           export = 'app_data/total_regs_card.csv')

+----+------+
|year| count|
+----+------+
|2022|362338|
|2023|368579|
+----+------+



The next cell executes a query that identifies the top brand for the current year based on registration volume, and gets the number of registrations for the current and previous year (again, to show year-over-year change).

In [13]:
query = f"""
SELECT year, brand, COUNT(*) AS count
FROM data
WHERE year IN ({curr_yr-1},{curr_yr}) AND month <= {curr_mo} AND brand IN 
    (SELECT brand 
    FROM
    (SELECT year, brand, COUNT(*) AS count
    FROM data
    WHERE year IN ({curr_yr})
    GROUP BY year, brand
    ORDER BY -count
    LIMIT 1))
GROUP BY year, brand
ORDER BY year
LIMIT 2
"""
top_brand_count_2yr = query_to_df(query, show_num = 2, 
                                  export = 'app_data/top_brand_card.csv')

+----+------+-----+
|year| brand|count|
+----+------+-----+
|2022|TOYOTA|59692|
|2023|TOYOTA|56984|
+----+------+-----+



To get the data for the third and final card, the next cell gets the registration volumes for all of the fuel categories for the previous two years. The card that uses this data will show the fuel category with the greatest year-over-year change (excluding the "Other" category).

In [14]:
query = f"""
SELECT year, 
       COUNT(*) AS total,
       SUM(gasoline) AS Gasoline,
       SUM(hybrid) AS Hybrid,
       SUM(electric) AS Electric,
       SUM(diesel) AS Diesel
FROM 
    (SELECT year, gasoline, hybrid, electric, diesel
    FROM data
    WHERE year IN ({curr_yr-1},{curr_yr}) AND month <= {curr_mo})
GROUP BY year
ORDER BY year
"""
fuel_types_curryr = query_to_df(query, show_num = 2, 
                                export = 'app_data/fuel_card.csv')

+----+------+--------+------+--------+------+
|year| total|Gasoline|Hybrid|Electric|Diesel|
+----+------+--------+------+--------+------+
|2022|362338|  283195| 33569|   18383| 14418|
|2023|368579|  270867| 41049|   33196| 12016|
+----+------+--------+------+--------+------+



## Top Brands Bar Chart Data

The dashboard will include a bar chart showing registration volumes for the top brands for different years and counties (as well as the overall statewide figures), based on user-selected options. The registration volumes will be disaggregated by fuel type. The below cell implements the query to get the data needed for this chart.

In [15]:
#query top brands by county
num_brands = 10
query = f"""
SELECT * , 
    CASE WHEN total>=1000 THEN CONCAT(ROUND(total/1000, 2),'K') 
    ELSE total END AS data_label FROM
((SELECT * 
FROM
    (SELECT year, county, brand, total, Gasoline, Hybrid, Electric, Diesel, Other,
           ROW_NUMBER() OVER (PARTITION BY year, county ORDER BY total DESC) AS rank
    FROM
        (SELECT year, 
           county,
           brand,
           COUNT(*) as total,
           SUM(gasoline) as Gasoline,
           SUM(hybrid) as Hybrid,
           SUM(electric) as Electric,
           SUM(diesel) as Diesel,
           SUM(other) as Other
        FROM 
            (SELECT year, county, brand, gasoline, hybrid, electric, diesel, other
            FROM data)
        GROUP BY year, county, brand))
WHERE rank<={num_brands})
UNION
(SELECT *
FROM
    (SELECT year, 'Statewide' AS county, brand, total, Gasoline, Hybrid, Electric, Diesel, Other,
           ROW_NUMBER() OVER (PARTITION BY year ORDER BY total DESC) AS rank
    FROM
        (SELECT year, 
           brand,
           COUNT(*) as total,
           SUM(gasoline) as Gasoline,
           SUM(hybrid) as Hybrid,
           SUM(electric) as Electric,
           SUM(diesel) as Diesel,
           SUM(other) as Other
        FROM 
            (SELECT year, brand, gasoline, hybrid, electric, diesel, other
            FROM data)
        GROUP BY year, brand))
WHERE rank<={num_brands}))
ORDER BY year, county, -total
"""
top_brands = query_to_df(query, show_num = 5, export = 'app_data/top_brands.csv')

+----+------+---------+-----+--------+------+--------+------+-----+----+----------+
|year|county|    brand|total|Gasoline|Hybrid|Electric|Diesel|Other|rank|data_label|
+----+------+---------+-----+--------+------+--------+------+-----+----+----------+
|2020| Adams|     FORD|  149|     122|     1|       0|    19|    7|   1|       149|
|2020| Adams|CHEVROLET|  130|     115|     0|       1|     5|    9|   2|       130|
|2020| Adams|    HONDA|  105|     100|     4|       0|     0|    1|   3|       105|
|2020| Adams|   TOYOTA|   95|      84|    10|       0|     0|    1|   4|        95|
|2020| Adams|   NISSAN|   57|      57|     0|       0|     0|    0|   5|        57|
+----+------+---------+-----+--------+------+--------+------+-----+----+----------+
only showing top 5 rows



## Monthly Time Series

The dashboard will also feature a monthly time series area chart. It will show monthly registrations by fuel type for user-specified brands (or all brands) and counties (or statewide). There will be an option to display the data in terms of registration volumes or as fuel shares such that the total value for each month is 100%. The below query pulls the data required for this chart.

In [16]:
#add county to above
query = f"""
SELECT * , ROUND(100*Gasoline_volume/Total_volume,2) AS Gasoline_share, 
           ROUND(100*Hybrid_volume/Total_volume,2) AS Hybrid_share,
           ROUND(100*Electric_volume/Total_volume,2) AS Electric_share, 
           ROUND(100*Diesel_volume/Total_volume,2) AS Diesel_share, 
           ROUND(100*Other_volume/Total_volume,2) AS Other_share
FROM
    ((SELECT year, month, 'Statewide' AS county, 'ALL BRANDS' AS brand, 
           COUNT(*) AS Total_volume,
           SUM(gasoline) AS Gasoline_volume,
           SUM(hybrid) AS Hybrid_volume,
           SUM(electric) AS Electric_volume,
           SUM(diesel) AS Diesel_volume,
           SUM(other) AS Other_volume
        FROM 
            (SELECT year, month, gasoline, hybrid, electric, diesel, other
            FROM data)
        GROUP BY year, month
        ORDER BY year, month)
    UNION
    (SELECT year, month, county, 'ALL BRANDS' AS brand, 
           COUNT(*) AS Total_volume,
           SUM(gasoline) AS Gasoline_volume,
           SUM(hybrid) AS Hybrid_volume,
           SUM(electric) AS Electric_volume,
           SUM(diesel) AS Diesel_volume,
           SUM(other) AS Other_volume
        FROM 
            (SELECT year, month, county, gasoline, hybrid, electric, diesel, other
            FROM data)
        GROUP BY year, month, county
        ORDER BY year, month, county)
    UNION
    (SELECT year, month, 'Statewide' AS county, brand, 
           COUNT(*) AS Total_volume,
           SUM(gasoline) AS Gasoline_volume,
           SUM(hybrid) AS Hybrid_volume,
           SUM(electric) AS Electric_volume,
           SUM(diesel) AS Diesel_volume,
           SUM(other) AS Other_volume
        FROM 
            (SELECT year, month, brand, gasoline, hybrid, electric, diesel, other
            FROM data
            WHERE brand IN {top_brands_curryr_str})
        GROUP BY year, month, brand
        ORDER BY year, month, brand)
    UNION
    (SELECT year, month, county, brand, 
           COUNT(*) AS Total,
           SUM(gasoline) AS Gasoline,
           SUM(hybrid) AS Hybrid,
           SUM(electric) AS Electric,
           SUM(diesel) AS Diesel,
           SUM(other) AS Other
        FROM 
            (SELECT year, month, county, brand, gasoline, hybrid, electric, diesel, other
            FROM data
            WHERE brand IN {top_brands_curryr_str})
        GROUP BY year, month, county, brand
        ORDER BY year, month, county, brand))
ORDER BY year, month, county, brand
"""
mnly_ts_data = query_to_df(query, show_num = 5)

+----+-----+------+----------+------------+---------------+-------------+---------------+-------------+------------+--------------+------------+--------------+------------+-----------+
|year|month|county|     brand|Total_volume|Gasoline_volume|Hybrid_volume|Electric_volume|Diesel_volume|Other_volume|Gasoline_share|Hybrid_share|Electric_share|Diesel_share|Other_share|
+----+-----+------+----------+------------+---------------+-------------+---------------+-------------+------------+--------------+------------+--------------+------------+-----------+
|2020|    1| Adams|ALL BRANDS|          95|             77|            3|              1|            8|           6|         81.05|        3.16|          1.05|        8.42|       6.32|
|2020|    1| Adams| CHEVROLET|          11|             10|            0|              0|            0|           1|         90.91|         0.0|           0.0|         0.0|       9.09|
|2020|    1| Adams|      FORD|          23|             20|            1|  

Before exporting the data, a few formatting adjustments are made to the data using Pandas in the below cell.

In [17]:
#format brand names
mnly_ts_data['brand'] = mnly_ts_data['brand'].apply(lambda x: get_formatted_brand_name(x))

#ensure all year, month, county, and brand combinations are represented in the data,
#even if they have zero values
yrs = sorted(mnly_ts_data['year'].unique())
mos = sorted(mnly_ts_data['month'].unique())
counties = sorted(mnly_ts_data['county'].unique())
brands = sorted(mnly_ts_data['brand'].unique())
all_combos_df = pd.DataFrame([i for i in product(yrs,mos,counties,brands)], 
                          columns = ['year','month','county','brand'])
mnly_ts_data = all_combos_df.merge(mnly_ts_data, how = 'left', 
                                   on = ['year','month','county','brand'])

#make a datetime format variable based on year and month columns
mnly_ts_data['dt'] = pd.to_datetime(mnly_ts_data[['year','month']].assign(day = 1))

#ensure all values are for current month and earlier, 
#since the all_combos_df will include all months for all years
timefilt = mnly_ts_data['dt']<pd.to_datetime(f'{curr_yr}-{curr_mo+1}-1')
mnly_ts_data = mnly_ts_data[timefilt]

#export
mnly_ts_data.to_csv('app_data/mnly_ts_data.csv', index = False)

/Users/gregtrunz/opt/anaconda3/envs/pyspark_env/lib/python3.8/site-packages/pandas/core/internals/blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


## Map and Scatterplot Data

Finally, we will get vehicle registration data that will be added on to the county-level data in `map_df` and used in the map and scatter plot visuals in the dashboard. The choropleth map will show registration data by county for 2023 and the scatterplot will allow users to examine how different brands and fuel types correlate with median income or population density across counties. 

The following code cell gets total registration volumes and volumes for each fuel type across all brands, and merges this data with `map_df`.

In [18]:
#counts by county and fuel type, all brands
query = f"""
SELECT county,
            SUM(total) AS {curr_yr}_allbrands_allfueltypes_count, 
            SUM(gasoline) AS {curr_yr}_allbrands_gasoline_count, 
            SUM(hybrid) AS {curr_yr}_allbrands_hybrid_count,
            SUM(electric) AS {curr_yr}_allbrands_electric_count, 
            SUM(diesel) AS {curr_yr}_allbrands_diesel_count
FROM
    (SELECT county,1 AS total,gasoline,hybrid,electric,diesel                    
    FROM data
    WHERE year={curr_yr})
GROUP BY county
ORDER BY county
"""
all_brand_df = query_to_df(query)
map_df = map_df.merge(all_brand_df, how = 'left', on = 'county')

The next cell gets total registration volumes and volumes for each fuel type for each of the top brands for the current year and merges this data with `map_df`.

In [19]:
#counts by county, year, brand
dfs = {}
for brand in top_brands_curryr['brand']:
    query = f"""
    SELECT county,
                SUM(total) AS {curr_yr}_{brand.lower()}_allfueltypes_count, 
                SUM(gasoline) AS {curr_yr}_{brand.lower()}_gasoline_count, 
                SUM(hybrid) AS {curr_yr}_{brand.lower()}_hybrid_count,
                SUM(electric) AS {curr_yr}_{brand.lower()}_electric_count, 
                SUM(diesel) AS {curr_yr}_{brand.lower()}_diesel_count
    FROM
        (SELECT county,1 AS total,gasoline,hybrid,electric,diesel                    
        FROM data
        WHERE year={curr_yr} AND brand='{brand}')
    GROUP BY county
    ORDER BY county
    """
    brand_df = query_to_df(query)
    dfs[f'{brand.lower()}'] = brand_df

#dfs together with each other and data from map_df
for i in dfs.values():
    map_df = map_df.merge(i, how = 'left', on = 'county')  

Finally, the next cell creates additional variables related to brand share, fuel type share, and registrations per capita based on the existing data in `map_df` and exports the data frame.

In [21]:
for brand in ['allbrands']+top_brands_curryr['brand'].str.lower().tolist():
    for ft in ['gasoline','hybrid','electric','diesel','allfueltypes']:
        shareft_den = map_df[f'{curr_yr}_{brand}_allfueltypes_count']
        sharebr_den = map_df[f'{curr_yr}_allbrands_{ft}_count']
        num = map_df[f'{curr_yr}_{brand}_{ft}_count']
        map_df[f'{curr_yr}_{brand}_{ft}_shareft'] = (100*num/ shareft_den).round(3)
        map_df[f'{curr_yr}_{brand}_{ft}_sharebr'] = (100*num/ sharebr_den).round(3)
        map_df[f'{curr_yr}_{brand}_{ft}_percap'] = (10000*num/map_df['population']).round(3)
        
map_df = map_df.fillna(0)
map_df.to_csv('app_data/map_data.csv', index = False)

In [22]:
spark.stop()